# 5주차: PCA와 차원 축소

## 학습 목표
- 차원의 저주를 이해하고 차원 축소의 필요성을 알아보자
- PCA(주성분 분석)의 직관적 원리를 이해하자
- StandardScaler로 데이터를 정규화하는 방법을 배우자
- 주성분을 해석하고 2D/3D로 시각화하자

## 1. 왜 차원을 줄여야 할까?

음원 데이터를 분석하다 보면 수많은 특징(feature)들을 다루게 된다.
템포, 에너지, 댄서빌리티, 밸런스, 어쿠스틱... 이런 특징이 10개, 20개, 심지어 100개가 넘을 수도 있다.

특징이 많으면 더 정확할 것 같지만, 실제로는 여러 문제가 발생한다:
- **시각화가 어렵다**: 3차원을 넘어가면 그래프로 표현할 수 없다
- **계산이 복잡하다**: 특징이 많을수록 처리 시간이 기하급수적으로 증가한다
- **과적합 위험**: 특징이 너무 많으면 오히려 성능이 떨어질 수 있다

이것을 '차원의 저주(Curse of Dimensionality)'라고 부른다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 기본 설정
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 2. 간단한 예시로 시작하기

### 2.1 2차원 데이터로 이해하기

먼저 눈으로 볼 수 있는 2차원 데이터로 PCA가 무엇을 하는지 직관적으로 이해해보자.
키와 몸무게 데이터를 예로 들어보겠다. 일반적으로 키가 크면 몸무게도 많이 나가는 경향이 있다.

In [ ]:
# 예시 데이터 생성: 키와 몸무게
np.random.seed(42)
n_samples = 100

# 키 (cm): 평균 170cm, 표준편차 10cm
height = np.random.normal(170, 10, n_samples)

# 몸무게 (kg): 키와 관련이 있도록 생성
# 대략적으로 몸무게 = (키 - 100) * 0.9 + 노이즈
weight = (height - 100) * 0.9 + np.random.normal(0, 5, n_samples)

# 데이터프레임 생성
data_2d = pd.DataFrame({
    'height': height,
    'weight': weight
})

print("📊 키와 몸무게 데이터 샘플:")
print(data_2d.head())
print(f"\n데이터 개수: {len(data_2d)}개")

# 시각화
plt.figure(figsize=(8, 6))
plt.scatter(data_2d['height'], data_2d['weight'], alpha=0.6, s=50)
plt.xlabel('Height (cm)')
plt.ylabel('Weight (kg)')
plt.title('Height vs Weight: Original Data')
plt.grid(True, alpha=0.3)
plt.show()

### 2.2 PCA 직접 체험해보기

PCA는 데이터의 **분산이 최대가 되는 방향**을 찾는다.
쉽게 말해, 데이터가 가장 넓게 퍼져있는 방향을 찾아서 그 방향을 새로운 축으로 삼는 것이다.

실제로 키-몸무게 데이터에 PCA를 적용해서 확인해보자!

In [ ]:
# 키-몸무게 데이터에 PCA 적용해보기
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 데이터 정규화 (PCA 전에 필수!)
scaler_2d = StandardScaler()
data_2d_scaled = scaler_2d.fit_transform(data_2d)

# PCA 적용
pca_2d = PCA(n_components=2)
data_2d_pca = pca_2d.fit_transform(data_2d_scaled)

# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 원본 데이터 (정규화된 버전)
axes[0].scatter(data_2d_scaled[:, 0], data_2d_scaled[:, 1], alpha=0.6, s=50, c='blue')
axes[0].set_xlabel('Height (standardized)')
axes[0].set_ylabel('Weight (standardized)')
axes[0].set_title('Original Data (Standardized)')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0].axvline(x=0, color='k', linestyle='-', linewidth=0.5)

# PCA 변환된 데이터
axes[1].scatter(data_2d_pca[:, 0], data_2d_pca[:, 1], alpha=0.6, s=50, c='red')
axes[1].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)')
axes[1].set_title('PCA Transformed Data')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[1].axvline(x=0, color='k', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# 결과 해석
print("🔍 PCA 결과:")
print(f"PC1이 전체 분산의 {pca_2d.explained_variance_ratio_[0]*100:.1f}%를 설명")
print(f"PC2가 전체 분산의 {pca_2d.explained_variance_ratio_[1]*100:.1f}%를 설명")
print(f"총 {sum(pca_2d.explained_variance_ratio_)*100:.1f}%의 정보를 보존")

print("\n💡 관찰:")
print("- 원본 데이터는 대각선 방향으로 퍼져있었습니다")
print("- PCA는 이 대각선을 새로운 X축(PC1)으로 회전시켰습니다")
print("- PC1 하나만으로도 대부분의 정보를 표현할 수 있습니다!")
print("- 만약 PC1만 사용한다면 2차원 → 1차원으로 차원 축소 가능")

## 3. 음원 데이터로 PCA 실습

### 3.1 음원 데이터 생성

이제 실제 음원 분석처럼 여러 특징을 가진 데이터로 PCA를 실습해보자.
Lesson 4에서 사용했던 것과 동일한 데이터를 사용한다.

In [ ]:
# 시드 설정으로 재현 가능한 결과
np.random.seed(42)
n_tracks = 1000

# 장르 정의
genres = ['Pop', 'Rock', 'Electronic', 'Classical', 'Jazz', 'Hip-Hop', 'Country']
genre_weights = [0.2, 0.18, 0.15, 0.12, 0.1, 0.15, 0.1]

# 기본 데이터 구조
music_data = pd.DataFrame({
    'track_id': [f'T{i:04d}' for i in range(n_tracks)],
    'genre': np.random.choice(genres, n_tracks, p=genre_weights),
    'year': np.random.randint(1990, 2024, n_tracks)
})

# 음향 특징들 (0-1 스케일)
features = ['tempo', 'energy', 'danceability', 'valence', 'acousticness', 
           'instrumentalness', 'liveness', 'speechiness', 'loudness']

# 장르별 특성을 반영한 특징 생성
for feature in features:
    music_data[feature] = 0.0

# 장르별 특성 정의
for genre in genres:
    mask = music_data['genre'] == genre
    count = mask.sum()
    
    if genre == 'Pop':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.65, 0.15, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(6, 3, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(7, 3, count)
        music_data.loc[mask, 'valence'] = np.random.beta(6, 4, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(3, 7, count)
    elif genre == 'Rock':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.7, 0.2, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(8, 2, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(5, 5, count)
        music_data.loc[mask, 'valence'] = np.random.beta(5, 5, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(2, 8, count)
        music_data.loc[mask, 'loudness'] = np.random.beta(7, 3, count)
    elif genre == 'Electronic':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.75, 0.15, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(7, 2, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(8, 2, count)
        music_data.loc[mask, 'valence'] = np.random.beta(6, 4, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(1, 9, count)
        music_data.loc[mask, 'instrumentalness'] = np.random.beta(6, 4, count)
    elif genre == 'Classical':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.4, 0.2, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(3, 7, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(2, 8, count)
        music_data.loc[mask, 'valence'] = np.random.beta(4, 6, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(9, 1, count)
        music_data.loc[mask, 'instrumentalness'] = np.random.beta(8, 2, count)
        music_data.loc[mask, 'liveness'] = np.random.beta(6, 4, count)
    elif genre == 'Jazz':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.55, 0.25, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(5, 5, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(4, 6, count)
        music_data.loc[mask, 'valence'] = np.random.beta(5, 5, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(7, 3, count)
        music_data.loc[mask, 'instrumentalness'] = np.random.beta(6, 4, count)
        music_data.loc[mask, 'liveness'] = np.random.beta(7, 3, count)
    elif genre == 'Hip-Hop':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.5, 0.15, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(6, 4, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(8, 2, count)
        music_data.loc[mask, 'valence'] = np.random.beta(4, 6, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(2, 8, count)
        music_data.loc[mask, 'speechiness'] = np.random.beta(7, 3, count)
        music_data.loc[mask, 'loudness'] = np.random.beta(6, 4, count)
    else:  # Country
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(0.6, 0.2, count), 0, 1)
        music_data.loc[mask, 'energy'] = np.random.beta(5, 5, count)
        music_data.loc[mask, 'danceability'] = np.random.beta(5, 5, count)
        music_data.loc[mask, 'valence'] = np.random.beta(6, 4, count)
        music_data.loc[mask, 'acousticness'] = np.random.beta(6, 4, count)

# 기본 특징이 없는 경우 랜덤 값으로 채우기
for feature in ['instrumentalness', 'liveness', 'speechiness', 'loudness']:
    mask = music_data[feature] == 0
    music_data.loc[mask, feature] = np.random.beta(3, 7, mask.sum())

print(f"🎵 음원 데이터셋 생성 완료!")
print(f"총 {n_tracks}개 트랙, {len(genres)}개 장르")
print(f"특징 수: {len(features)}개")
print(f"데이터 형태: {music_data.shape}")

print("\n📊 데이터 샘플:")
print(music_data.head())

print("\n📈 장르별 곡 수:")
print(music_data['genre'].value_counts())

### 3.2 데이터 확인

PCA를 적용하기 전에 데이터의 기본 정보만 간단히 확인하자.
(자세한 EDA는 이미 4주차에서 다뤘으므로 여기서는 PCA에 집중한다)

In [ ]:
# 특징들의 기본 통계
print("📊 각 특징의 기본 통계:")
print(music_data[features].describe().round(3))

print("\n💡 문제 상황:")
print("- 9개의 특징을 한 번에 시각화하기 어렵다")
print("- 특징들 간의 관계를 전체적으로 파악하기 힘들다")
print("- 장르별 차이를 9차원 공간에서 보기는 불가능하다")
print("\n👉 해결책: PCA로 차원을 줄여서 전체 구조를 한눈에 보자!")

## 4. 데이터 정규화 - StandardScaler

### 4.1 왜 정규화가 필요한가?

PCA를 적용하기 전에 매우 중요한 단계가 있다: **데이터 정규화(Standardization)**

예를 들어:
- 템포: 60 ~ 200 BPM (큰 범위)
- 에너지: 0.0 ~ 1.0 (작은 범위)

이런 경우 PCA는 단순히 값이 큰 템포만 중요하다고 생각할 수 있다.
모든 특징을 공평하게 비교하려면 같은 스케일로 맞춰줘야 한다.

In [ ]:
# 음악 특징만 선택
X = music_data[features].values
print("원본 데이터 shape:", X.shape)
print(f"(곡 {X.shape[0]}개, 특징 {X.shape[1]}개)\n")

# 정규화 전 데이터 확인
print("📊 정규화 전 데이터 통계:")
print(f"평균: {X.mean(axis=0).round(3)}")
print(f"표준편차: {X.std(axis=0).round(3)}")
print(f"최솟값: {X.min(axis=0).round(3)}")
print(f"최댓값: {X.max(axis=0).round(3)}")

# StandardScaler 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n📊 정규화 후 데이터 통계:")
print(f"평균: {X_scaled.mean(axis=0).round(3)}")
print(f"표준편차: {X_scaled.std(axis=0).round(3)}")
print(f"최솟값: {X_scaled.min(axis=0).round(3)}")
print(f"최댓값: {X_scaled.max(axis=0).round(3)}")

print("\n✅ 정규화 완료!")
print("   - 평균이 0에 가까워졌습니다")
print("   - 표준편차가 1에 가까워졌습니다")
print("   - 이제 모든 특징이 같은 스케일을 가집니다")

### 4.2 정규화 효과 시각화

In [ ]:
# 정규화 전후 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 정규화 전
axes[0].boxplot(X, labels=features)
axes[0].set_title('Before Standardization')
axes[0].set_ylabel('Value')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# 정규화 후
axes[1].boxplot(X_scaled, labels=features)
axes[1].set_title('After Standardization')
axes[1].set_ylabel('Standardized Value')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5, label='Mean = 0')
axes[1].legend()

plt.tight_layout()
plt.show()

print("💡 정규화의 효과:")
print("   - 모든 특징이 비슷한 범위를 가지게 되었습니다")
print("   - 평균이 0 근처로 이동했습니다")
print("   - 이제 PCA가 모든 특징을 공평하게 고려할 수 있습니다")

## 5. PCA 적용하기

### 5.1 PCA 실행

드디어 PCA를 적용할 차례다! 9차원 데이터를 2차원으로 줄여보자.

In [ ]:
# PCA 객체 생성 (2차원으로 축소)
pca = PCA(n_components=2)

# PCA 적용
X_pca = pca.fit_transform(X_scaled)

print("🎯 PCA 완료!")
print(f"원본 데이터: {X_scaled.shape} (곡 {X_scaled.shape[0]}개 × 특징 {X_scaled.shape[1]}개)")
print(f"변환된 데이터: {X_pca.shape} (곡 {X_pca.shape[0]}개 × 주성분 {X_pca.shape[1]}개)")
print("\n9차원 → 2차원으로 축소되었습니다! 🎉")

# 설명된 분산 비율
explained_variance = pca.explained_variance_ratio_
print(f"\n📊 각 주성분이 설명하는 정보의 양:")
print(f"   PC1 (첫 번째 주성분): {explained_variance[0]*100:.1f}%")
print(f"   PC2 (두 번째 주성분): {explained_variance[1]*100:.1f}%")
print(f"   총합: {sum(explained_variance)*100:.1f}%")

print(f"\n💡 해석:")
print(f"   원본 데이터의 {sum(explained_variance)*100:.1f}%를 단 2개의 차원으로 표현할 수 있습니다!")
if sum(explained_variance) > 0.6:
    print("   꽤 좋은 압축률입니다! 👍")
else:
    print("   정보 손실이 좀 있지만, 시각화에는 충분합니다.")

### 5.2 주성분 이해하기

각 주성분(PC)이 무엇을 의미하는지 알아보자.
주성분은 원래 특징들의 조합으로 만들어진다.

In [ ]:
# 주성분의 구성 (로딩)
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=features
)

print("🔍 각 주성분을 구성하는 원래 특징들의 기여도:")
print(loadings.round(3))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PC1 구성
colors1 = ['green' if x > 0 else 'red' for x in loadings['PC1']]
axes[0].barh(loadings.index, loadings['PC1'], color=colors1, alpha=0.7)
axes[0].set_xlabel('Loading')
axes[0].set_title('PC1 Composition')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# PC2 구성
colors2 = ['green' if x > 0 else 'red' for x in loadings['PC2']]
axes[1].barh(loadings.index, loadings['PC2'], color=colors2, alpha=0.7)
axes[1].set_xlabel('Loading')
axes[1].set_title('PC2 Composition')
axes[1].grid(True, alpha=0.3)
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# 주성분 해석
print("\n💡 주성분 해석:")
print("\nPC1 (첫 번째 주성분):")
top_pc1 = loadings['PC1'].abs().nlargest(3)
for feature in top_pc1.index:
    value = loadings.loc[feature, 'PC1']
    direction = "높을수록" if value > 0 else "낮을수록"
    print(f"  - {feature}: {direction} PC1이 높아짐 (기여도: {abs(value):.3f})")

print("\nPC2 (두 번째 주성분):")
top_pc2 = loadings['PC2'].abs().nlargest(3)
for feature in top_pc2.index:
    value = loadings.loc[feature, 'PC2']
    direction = "높을수록" if value > 0 else "낮을수록"
    print(f"  - {feature}: {direction} PC2가 높아짐 (기여도: {abs(value):.3f})")

## 6. PCA 결과 시각화

### 6.1 2D 산점도

이제 9차원 데이터를 2차원 평면에 표현할 수 있다!

In [ ]:
# PCA 결과를 데이터프레임으로 정리
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
pca_df['genre'] = music_data['genre'].values
pca_df['track_id'] = music_data['track_id'].values

# 전체 장르 시각화
plt.figure(figsize=(10, 8))

# 장르별로 다른 색상과 마커로 표시
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']
markers = ['o', 's', '^', 'D', 'v', 'h', 'p']

for i, genre in enumerate(genres):
    genre_data = pca_df[pca_df['genre'] == genre]
    plt.scatter(genre_data['PC1'], genre_data['PC2'], 
               c=colors[i], marker=markers[i], label=genre,
               alpha=0.6, s=50)

plt.xlabel(f'PC1 ({explained_variance[0]*100:.1f}% of variance)')
plt.ylabel(f'PC2 ({explained_variance[1]*100:.1f}% of variance)')
plt.title('Music Data in 2D PCA Space')
plt.legend(title='Genre', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

# 원점에서 십자선 추가
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

print("🎯 관찰 포인트:")
print("1. 같은 장르의 곡들이 비슷한 위치에 모여있나요?")
print("2. 어떤 장르들이 서로 가깝게 위치하나요?")
print("3. 특별히 떨어져 있는 곡들이 있나요? (이상치)")

### 6.2 선택적 장르 비교

In [ ]:
# 선택된 장르만 따로 보기
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 전체 장르 (작게)
for i, genre in enumerate(genres):
    genre_data = pca_df[pca_df['genre'] == genre]
    axes[0].scatter(genre_data['PC1'], genre_data['PC2'], 
                   c=colors[i], marker=markers[i], label=genre,
                   alpha=0.4, s=20)

axes[0].set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}% of variance)')
axes[0].set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}% of variance)')
axes[0].set_title('All Genres (Overview)')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linestyle='--', linewidth=0.5)
axes[0].axvline(x=0, color='k', linestyle='--', linewidth=0.5)

# 대비가 강한 장르들만 확대
selected_genres = ['Classical', 'Electronic', 'Hip-Hop']
for genre in selected_genres:
    genre_data = pca_df[pca_df['genre'] == genre]
    color_idx = genres.index(genre)
    axes[1].scatter(genre_data['PC1'], genre_data['PC2'], 
                   c=colors[color_idx], marker=markers[color_idx], 
                   label=genre, alpha=0.7, s=60)

axes[1].set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}% of variance)')
axes[1].set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}% of variance)')
axes[1].set_title('Contrasting Genres')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linestyle='--', linewidth=0.5)
axes[1].axvline(x=0, color='k', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

print("💡 대비 분석:")
print("- Classical: 주로 왼쪽 (어쿠스틱, 낮은 에너지)")
print("- Electronic: 주로 오른쪽 (높은 에너지, 댄서빌리티)")
print("- Hip-Hop: 중간 영역 (특별한 speechiness 특성)")

## 7. 최적 주성분 개수 찾기

### 7.1 Scree Plot

몇 개의 주성분을 사용하는 것이 적절한지 판단하는 방법을 알아보자.

In [ ]:
# 모든 주성분 계산
pca_full = PCA()
pca_full.fit(X_scaled)

# 각 주성분의 설명 분산
explained_var = pca_full.explained_variance_ratio_
cumsum_var = np.cumsum(explained_var)

# Scree Plot과 누적 설명 분산
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree Plot
axes[0].bar(range(1, len(explained_var)+1), explained_var, alpha=0.7, color='blue')
axes[0].plot(range(1, len(explained_var)+1), explained_var, 'ro-', linewidth=2)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Scree Plot')
axes[0].set_xticks(range(1, len(explained_var)+1))
axes[0].grid(True, alpha=0.3)

# 누적 설명 분산
axes[1].plot(range(1, len(cumsum_var)+1), cumsum_var, 'bo-', linewidth=2)
axes[1].axhline(y=0.8, color='r', linestyle='--', label='80% threshold')
axes[1].axhline(y=0.9, color='orange', linestyle='--', label='90% threshold')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Explained Variance')
axes[1].set_xticks(range(1, len(cumsum_var)+1))
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 권장 주성분 개수
n_80 = np.argmax(cumsum_var >= 0.8) + 1
n_90 = np.argmax(cumsum_var >= 0.9) + 1

print("📊 주성분 개수 선택 가이드:")
print(f"   - 80% 설명하려면: {n_80}개 주성분 필요")
print(f"   - 90% 설명하려면: {n_90}개 주성분 필요")
print(f"\n현재 우리가 사용한 개수:")
print(f"   - 2D: {sum(explained_variance)*100:.1f}% 설명")

print("\n💡 팁:")
print("   - 시각화 목적: 2-3개면 충분")
print("   - 머신러닝 목적: 80-90% 설명할 수 있는 개수 선택")
print("   - Elbow point: 그래프가 꺾이는 지점도 좋은 선택")

## 8. PCA 활용 예시

### 8.1 유사한 곡 찾기

PCA 공간에서 가까운 곡들은 음악적 특성이 비슷하다.

In [ ]:
def find_similar_songs(song_id, pca_df, n=5):
    """PCA 공간에서 가장 비슷한 곡 찾기"""
    
    # 타겟 곡의 좌표
    target = pca_df[pca_df['track_id'] == song_id]
    
    if len(target) == 0:
        print(f"Song {song_id} not found!")
        return None
    
    target_pc1 = target['PC1'].values[0]
    target_pc2 = target['PC2'].values[0]
    target_genre = target['genre'].values[0]
    
    # 모든 곡과의 거리 계산
    distances = np.sqrt((pca_df['PC1'] - target_pc1)**2 + 
                       (pca_df['PC2'] - target_pc2)**2)
    
    # 자기 자신 제외하고 가장 가까운 곡들 찾기
    pca_df_copy = pca_df.copy()
    pca_df_copy['distance'] = distances
    similar = pca_df_copy[pca_df_copy['track_id'] != song_id].nsmallest(n, 'distance')
    
    print(f"🎵 '{song_id}' (장르: {target_genre})와 비슷한 곡들:")
    print("-" * 50)
    for idx, row in similar.iterrows():
        print(f"  {row['track_id']} (장르: {row['genre']}, 거리: {row['distance']:.3f})")
    
    return similar, target

# 예시: 첫 번째 곡과 비슷한 곡 찾기
similar_songs, target_song = find_similar_songs('T0708', pca_df, n=5)

# 시각화
if similar_songs is not None:
    plt.figure(figsize=(10, 8))
    
    # 모든 곡들 (회색)
    plt.scatter(pca_df['PC1'], pca_df['PC2'], alpha=0.2, s=30, c='gray')
    
    # 타겟 곡 (빨강)
    plt.scatter(target_song['PC1'], target_song['PC2'], s=200, c='red', 
               marker='*', edgecolor='black', linewidth=2, label='Target Song')
    
    # 비슷한 곡들 (파랑)
    plt.scatter(similar_songs['PC1'], similar_songs['PC2'], s=100, c='blue',
               alpha=0.8, label='Similar Songs')
    
    # 연결선 그리기
    for idx, row in similar_songs.iterrows():
        plt.plot([target_song['PC1'].values[0], row['PC1']], 
                [target_song['PC2'].values[0], row['PC2']],
                'b--', alpha=0.3, linewidth=1)
    
    plt.xlabel(f'PC1 ({explained_variance[0]*100:.1f}% of variance)')
    plt.ylabel(f'PC2 ({explained_variance[1]*100:.1f}% of variance)')
    plt.title('Finding Similar Songs in PCA Space')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.show()

print("\n💡 응용:")
print("   - 음악 추천 시스템의 기초")
print("   - 플레이리스트 자동 생성")
print("   - 장르 크로스오버 곡 발견")

### 8.2 장르별 중심점 찾기

PCA 공간에서 각 장르의 중심점을 찾아서 장르별 특성을 요약해보자.

In [ ]:
# 장르별 중심점(centroid) 계산
genre_centroids = pca_df.groupby('genre')[['PC1', 'PC2']].mean()
print("🎯 장르별 PCA 공간에서의 중심점:")
print(genre_centroids.round(3))

# 원점에서 가장 먼 장르와 가까운 장르 찾기
genre_centroids['distance_from_origin'] = np.sqrt(
    genre_centroids['PC1']**2 + genre_centroids['PC2']**2
)

print(f"\n📏 원점에서의 거리:")
for genre in genre_centroids.index:
    distance = genre_centroids.loc[genre, 'distance_from_origin']
    print(f"  {genre}: {distance:.3f}")

# 시각화
plt.figure(figsize=(12, 8))

# 모든 곡들 (반투명)
for i, genre in enumerate(genres):
    genre_data = pca_df[pca_df['genre'] == genre]
    plt.scatter(genre_data['PC1'], genre_data['PC2'], 
               c=colors[i], marker=markers[i], label=genre,
               alpha=0.3, s=30)

# 장르별 중심점 (크고 진하게)
for i, genre in enumerate(genres):
    centroid = genre_centroids.loc[genre]
    plt.scatter(centroid['PC1'], centroid['PC2'], 
               c=colors[i], marker=markers[i], 
               s=300, alpha=1.0, edgecolor='black', linewidth=2,
               label=f'{genre} Center')

# 원점에서 중심점까지 화살표
for i, genre in enumerate(genres):
    centroid = genre_centroids.loc[genre]
    plt.arrow(0, 0, centroid['PC1'], centroid['PC2'], 
              head_width=0.15, head_length=0.1, fc=colors[i], ec=colors[i], 
              alpha=0.7, linewidth=2)

plt.xlabel(f'PC1 ({explained_variance[0]*100:.1f}% of variance)')
plt.ylabel(f'PC2 ({explained_variance[1]*100:.1f}% of variance)')
plt.title('Genre Centroids in PCA Space')

# 범례는 중심점만
legend_elements = [plt.scatter([], [], c=colors[i], marker=markers[i], s=100, 
                              label=f'{genre}') for i, genre in enumerate(genres)]
plt.legend(handles=legend_elements, title='Genre Centers', 
          bbox_to_anchor=(1.05, 1), loc='upper left')

plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

# 가장 극단적인 장르들
most_extreme = genre_centroids['distance_from_origin'].idxmax()
least_extreme = genre_centroids['distance_from_origin'].idxmin()

print(f"\n🔍 분석 결과:")
print(f"   - 가장 극단적 특성: {most_extreme} (원점에서 거리: {genre_centroids.loc[most_extreme, 'distance_from_origin']:.3f})")
print(f"   - 가장 평균적 특성: {least_extreme} (원점에서 거리: {genre_centroids.loc[least_extreme, 'distance_from_origin']:.3f})")
print(f"\n💡 해석:")
print(f"   - {most_extreme}는 다른 장르들과 가장 구별되는 특성을 가집니다")
print(f"   - {least_extreme}는 전체 음악의 평균적 특성에 가깝습니다")
print(f"   - 화살표 방향이 각 장르의 특성을 나타냅니다")